# Find out if the person has chances to recover fully after 6 months

### Required Data
'RCONSC', 'SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RVISINF', 'RSBP', 'RDEF1', 'RDEF2', 'RDEF3',
'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'STYPE', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK', 'FRECOVER'

## Steps

- process input
- process output
- split test and train data
- train model
- test model
- save model

In [25]:
import pandas as pd
import numpy as np
import timeit
from sklearn import svm
from sklearn.cross_validation import train_test_split
import pickle

In [20]:
data = pd.read_csv('data.CSV')
# preprocessing the dataset
# selecting required fields

required_fields = ['RCONSC', 'SEX', 'AGE', 'RSLEEP', 'RATRIAL', 'RVISINF', 'RSBP', 'RDEF1', 'RDEF2', 'RDEF3',
                  'RDEF4', 'RDEF5', 'RDEF6', 'RDEF7', 'RDEF8', 'STYPE', 'DDIAGISC', 'DDIAGHA', 'DDIAGUN', 'DNOSTRK'
                  , 'FRECOVER']
data = data[required_fields]
data.dropna() # dropping fields with NA values or Empty Values
data = data[data['FRECOVER'] != 'U']
data.to_pickle('doa-data/predictFullRecovery.pkl') # saving to file
data.to_csv('doa-data/predictFullRecovery.csv') # saving to file

In [21]:
## processing INPUTS AND OUTPUTS

data_d = pd.get_dummies(data)
print data_d.columns
x = data_d.loc[:, 'AGE':'STYPE_TACS'].values # INPUT DATA READY

## PREPARING OUTPUT ARRAY
output_data = data['FRECOVER']
output_data = output_data.values
y = [1 if i == 'Y' else -1 for i in output_data] # if the input is Y then change to 1 else change to -1

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=2)
# 30% data is now test data (x_test, y_test) x is input y is output
# x_train and y_train are training data 70%

Index([u'AGE', u'RSBP', u'RCONSC_D', u'RCONSC_F', u'RCONSC_U', u'SEX_F',
       u'SEX_M', u'RSLEEP_N', u'RSLEEP_Y', u'RATRIAL_N', u'RATRIAL_Y',
       u'RVISINF_N', u'RVISINF_Y', u'RDEF1_C', u'RDEF1_N', u'RDEF1_Y',
       u'RDEF2_C', u'RDEF2_N', u'RDEF2_Y', u'RDEF3_C', u'RDEF3_N', u'RDEF3_Y',
       u'RDEF4_C', u'RDEF4_N', u'RDEF4_Y', u'RDEF5_C', u'RDEF5_N', u'RDEF5_Y',
       u'RDEF6_C', u'RDEF6_N', u'RDEF6_Y', u'RDEF7_C', u'RDEF7_N', u'RDEF7_Y',
       u'RDEF8_C', u'RDEF8_N', u'RDEF8_Y', u'STYPE_LACS', u'STYPE_OTH',
       u'STYPE_PACS', u'STYPE_POCS', u'STYPE_TACS', u'DDIAGISC_N',
       u'DDIAGISC_U', u'DDIAGISC_Y', u'DDIAGISC_u', u'DDIAGHA_N', u'DDIAGHA_U',
       u'DDIAGHA_Y', u'DDIAGUN_N', u'DDIAGUN_U', u'DDIAGUN_Y', u'DNOSTRK_N',
       u'DNOSTRK_U', u'DNOSTRK_Y', u'FRECOVER_N', u'FRECOVER_Y'],
      dtype='object')


In [22]:
# training our classifier
clf = svm.SVC()
print "starting training"
start_time = timeit.default_timer()
model = clf.fit(x_train, y_train)
print "training completed in %s seconds"  % (timeit.default_timer() - start_time)

starting training
training completed in 30.6368069649 seconds


In [23]:
#testing our classifier
print "starting testing"
start_time = timeit.default_timer()
print "Accuracy: %s precent" %(model.score(x_train, y_train) * 100)
print "testing completed in %s seconds"  % (timeit.default_timer() - start_time)

starting testing
Accuracy: 80.7386490302 precent
testing completed in 14.8410460949 seconds


In [26]:
# saving the model to file
pickle.dump(model, open('doa-data/svmFullRecoveryPredictor.pkl', 'wb'))